In [1]:
import PIL
import cPickle
import time
import numpy
import matplotlib
import tensorflow
import glob
import os
import math
import pp
import multiprocessing
%matplotlib inline

print "Packages loaded"

Packages loaded


In [2]:
ppservers = ()

job_server = pp.Server(ppservers=ppservers)

print "Wiht ",job_server.get_ncpus(),"workers"

Wiht  8 workers


In [3]:
direc = '/home/jerry/Downloads/notebook/KIOST/'

path1 = direc + 'train2/'
path2 = direc + 'trainannot/'
trainimglist = glob.glob(path1 + '/*.jpg')
trainimglist.sort()
trainannotlist = glob.glob(path2 + '/*.jpg')
trainannotlist.sort()
print ("%d train images" % (len(trainimglist)))
print ("%d train annotations" % (len(trainannotlist)))
       

16641 train images
16641 train annotations


In [4]:
height = 128
width = 128
nrclass = 3
trainData = None
trainLabel = None
trainLabelOneHot = None
trainlen = len(trainimglist)

In [5]:
def DenseToOneHot(labels_dense, num_classes):
    # Convert class labels from scalars to one-hot vectors.
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [6]:
def r2g(x, h, w):
        # Train image
        img1 = PIL.Image.open(x)
        img1 = img1.resize((h, w))
        gray  = numpy.array(img1).reshape(1, h, w, 3)
        gray = gray/255

        return gray

def l2l(x, h, w):
        img2 = PIL.Image.open(x)
        img2 = img2.resize((h, w), PIL.Image.NEAREST)
        label = numpy.array(img2).reshape(1, h, w, 1)
        label = label/255
       
        return label

In [7]:
def concat(x, y):
    x = np.concatenate((x, y), axis=0)
    return x

In [9]:
start_time = time.time()

gray = [(out, job_server.submit(r2g, (out, height, width), (),("PIL.Image","numpy",))) for out in trainimglist, trainannotlist, range(trainlen)]
label = job_server.submit(l2l, (f2, height, width), (),("PIL.Image","numpy",))


for (f1, f2, i) in zip(trainimglist, trainannotlist, range(trainlen)):
    
    # Stack images and labels
    if i == 0: 
        trainData =gray()
        trainLabel = label()
    else:
        trainData = numpy.concatenate((trainData, gray()), axis=0)
        trainLabel = numpy.concatenate((trainLabel, label()), axis=0)
        
print "time : ", time.time()-start_time

time :  1728.5681169


In [ ]:
start_time = time.time()
for (f1, f2, i) in zip(trainimglist, trainannotlist, range(trainlen)):
    # print (\"[%02d/%02d]f1: %sf2: %s\" % (i, trainlen, f1, f2))
   
    # Train image
    img1 = PIL.Image.open(f1)
    img1 = img1.resize((height, width))
    gray  = numpy.array(img1).reshape(1, height, width, 3)
    gray = gray/255
   
    # Train label
    img2 = PIL.Image.open(f2)
    img2 = img2.resize((height, width), PIL.Image.NEAREST)
    label = numpy.array(img2).reshape(1, height, width, 1)
    label = label/255
   
    # Stack images and labels
    if i == 0: 
        trainData =gray
        trainLabel = label
    else:
        trainData = numpy.concatenate((trainData, gray), axis=0)
        trainLabel = numpy.concatenate((trainLabel, label), axis=0)
        
print "time : ", time.time() - start_time

In [ ]:
# Onehot-coded label
    trainLabelOneHot = np.zeros((trainLabel.shape[0], trainLabel.shape[1], trainLabel.shape[2], nrclass))
    for row in range(height):
        for col in range(width):
            single = trainLabel[:, row, col, 0]
            oneHot = DenseToOneHot(single, nrclass) # (367,) => (367, 22)
            trainLabelOneHot[:, row, col, :] = oneHot
    print ("Train data process done.")